In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os


import matplotlib.pyplot as plt
import matplotlib.colors as colors

import numpy as np
from landlab.components import FlowDirectorSteepest, NetworkSedimentTransporter
from landlab.data_record import DataRecord
from landlab.grid.network import NetworkModelGrid
from landlab.plot import plot_network_and_parcels
from landlab.io import read_shapefile


from matplotlib.colors import Normalize

Create and run the synthetic example. 

In [ ]:
y_of_node = (0, 100, 200, 200, 300, 400, 400, 125)
x_of_node = (0, 0, 100, -50, -100, 50, -150, -100)

nodes_at_link = ((1, 0), (2, 1), (1, 7), (3, 1), (3, 4), (4, 5), (4, 6))

grid1 = NetworkModelGrid((y_of_node, x_of_node), nodes_at_link)
grid1.at_node["topographic__elevation"] = [0.0, 0.08, 0.25, 0.15, 0.25, 0.4, 0.8, 0.8]
grid1.at_node["bedrock__elevation"] = [0.0, 0.08, 0.25, 0.15, 0.25, 0.4, 0.8, 0.8]
grid1.at_link["flow_depth"] = 2.5 * np.ones(grid1.number_of_links) 
grid1.at_link["channel_slope"] = np.ones(grid1.number_of_links) # AP ?? Does it get overwritten?? check
grid1.at_link["reach_length"] = 200*np.ones(grid1.number_of_links)  # m
grid1.at_link["channel_width"] = 1*np.ones(grid1.number_of_links)

# element_id is the link on which the parcel begins. 
element_id = np.repeat(np.arange(grid1.number_of_links),30)
element_id = np.expand_dims(element_id, axis=1)

volume = 0.05*np.ones(np.shape(element_id))  # (m3)
active_layer = np.ones(np.shape(element_id)) # 1= active, 0 = inactive
density = 2650 * np.ones(np.size(element_id))  # (kg/m3)
abrasion_rate = 0 * np.ones(np.size(element_id)) # (mass loss /m)

# Lognormal GSD
medianD = 0.085 # m
mu = np.log(medianD)
sigma = np.log(2) #assume that D84 = sigma*D50
np.random.seed(0)
D = np.random.lognormal(
    mu,
    sigma,
    np.shape(element_id)
)  # (m) the diameter of grains in each parcel

time_arrival_in_link = np.random.rand(np.size(element_id), 1) 
location_in_link = np.random.rand(np.size(element_id), 1) 

lithology = ["quartzite"] * np.size(element_id)

variables = {
    "abrasion_rate": (["item_id"], abrasion_rate),
    "density": (["item_id"], density),
    "lithology": (["item_id"], lithology),
    "time_arrival_in_link": (["item_id", "time"], time_arrival_in_link),
    "active_layer": (["item_id", "time"], active_layer),
    "location_in_link": (["item_id", "time"], location_in_link),
    "D": (["item_id", "time"], D),
    "volume": (["item_id", "time"], volume)
}

items = {"grid_element": "link", "element_id": element_id}

_OUT_OF_NETWORK = NetworkModelGrid.BAD_INDEX - 1

parcels1 = DataRecord(
    grid1,
    items=items,
    time=[0.0],
    data_vars=variables,
    dummy_elements={"link": [_OUT_OF_NETWORK]},
)

fd1 = FlowDirectorSteepest(grid1, "topographic__elevation")
fd1.run_one_step()

nst1 = NetworkSedimentTransporter(    
    grid1,
    parcels1,
    fd1,
    bed_porosity=0.3,
    g=9.81,
    fluid_density=1000,
    transport_method="WilcockCrowe",
)
timesteps = 10 # total number of timesteps
dt = 60 * 60 * 24 *1 # length of timestep (seconds) 
for t in range(0, (timesteps * dt), dt):
    nst1.run_one_step(dt)

And now the shapefile example. 

In [ ]:
DATA_DIR = "./data/"
file = os.path.join(DATA_DIR, "MethowSubBasin.shp")

points_shapefile = os.path.join(DATA_DIR, "MethowSubBasin_Nodes_4.shp")
grid2 = read_shapefile(
    file,
    points_shapefile=points_shapefile,
    node_fields=["usarea_km2", "Elev_m"],
    link_fields=["usarea_km2", "Length_m"],
    link_field_conversion={"usarea_km2": "drainage_area", "Slope":"channel_slope", "Length_m":"reach_length"},
    node_field_conversion={
        "usarea_km2": "drainage_area",
        "Elev_m": "topographic__elevation",
    },
    threshold=0.01,
    )
grid2.at_node["bedrock__elevation"] = grid2.at_node["topographic__elevation"].copy()
grid2.at_link["channel_width"] = 1 * np.ones(grid2.number_of_links)
grid2.at_link["flow_depth"] = 0.5 * np.ones(grid2.number_of_links)

# element_id is the link on which the parcel begins. 
element_id = np.repeat(np.arange(grid2.number_of_links), 50)
element_id = np.expand_dims(element_id, axis=1)

volume = 1*np.ones(np.shape(element_id))  # (m3)
active_layer = np.ones(np.shape(element_id)) # 1= active, 0 = inactive
density = 2650 * np.ones(np.size(element_id))  # (kg/m3)
abrasion_rate = 0 * np.ones(np.size(element_id)) # (mass loss /m)

# Lognormal GSD
medianD = 0.15 # m
mu = np.log(medianD)
sigma = np.log(2) #assume that D84 = sigma*D50
np.random.seed(0)
D = np.random.lognormal(
    mu,
    sigma,
    np.shape(element_id)
)  # (m) the diameter of grains in each parcel

time_arrival_in_link = np.random.rand(np.size(element_id), 1) 
location_in_link = np.random.rand(np.size(element_id), 1) 

lithology = ["quartzite"] * np.size(element_id)

variables = {
    "abrasion_rate": (["item_id"], abrasion_rate),
    "density": (["item_id"], density),
    "lithology": (["item_id"], lithology),
    "time_arrival_in_link": (["item_id", "time"], time_arrival_in_link),
    "active_layer": (["item_id", "time"], active_layer),
    "location_in_link": (["item_id", "time"], location_in_link),
    "D": (["item_id", "time"], D),
    "volume": (["item_id", "time"], volume)
}

items = {"grid_element": "link", "element_id": element_id}

_OUT_OF_NETWORK = NetworkModelGrid.BAD_INDEX - 1

parcels2 = DataRecord(
    grid2,
    items=items,
    time=[0.0],
    data_vars=variables,
    dummy_elements={"link": [_OUT_OF_NETWORK]},
)

fd2 = FlowDirectorSteepest(grid2, "topographic__elevation")
fd2.run_one_step()

nst2 = NetworkSedimentTransporter(    
    grid2,
    parcels2,
    fd2,
    bed_porosity=0.3,
    g=9.81,
    fluid_density=1000,
    transport_method="WilcockCrowe",
)

for t in range(0, (timesteps * dt), dt):
    nst2.run_one_step(dt)
    
    print("Model time: ", t/(60*60*24), "days passed")

In [ ]:
network_norm = Normalize(-1, 6)
 
link_color_options = [
    {# empty dictionary = defaults
    },
    {
        "network_color":'r', # specify some simple modifications. 
        "network_linewidth":.7,
        "parcel_alpha":0
    },
    {
        "link_attribute": "sediment_total_volume", # use a link attribute
        "parcel_alpha":0
    },
    {
        "link_attribute": "sediment_total_volume", 
        "network_cmap": "jet", # change colormap
        "network_norm": network_norm, # and normalize
        "link_attribute_title": "Total Sediment Volume",
        "parcel_alpha":0,
        "network_linewidth":3

    }
]

for grid, parcels in zip([grid1, grid2], [parcels1, parcels2]):
    for l_opts in link_color_options:
        fig = plot_network_and_parcels(
            grid, parcels, 
            parcel_time_index=0, **l_opts)
        plt.show()
        

In [ ]:
random_link = np.random.randn(grid2.size("link"))

l_opts =    {
        "link_attribute": random_link, # use an array of size link. 
        "network_cmap": "jet", # change colormap
        "network_norm": network_norm, # and normalize
        "link_attribute_title": "A random number",
        "parcel_alpha":0,
        "network_linewidth":3

    }
fig = plot_network_and_parcels(
        grid2, parcels2, 
        parcel_time_index=0, **l_opts)
plt.show()

In [ ]:
parcel_color_norm = Normalize(0, 1)
parcel_color_norm2=colors.LogNorm(vmin=0.01, vmax=1)

parcel_color_options = [
    {# empty dictionary = defaults
    },
    {
        "parcel_color":'r', # specify some simple modifications. 
         "parcel_size":10
    },
    {
        "parcel_color_attribute": "D", # use a parcel attribute. 
        "parcel_color_norm": parcel_color_norm,
        "parcel_color_attribute_title":"Diameter [m]",
        "parcel_alpha":1.0,
    },
    {
        "parcel_color_attribute": "abrasion_rate", # an 
        "parcel_color_cmap": "bone",
    },
]

#add parcel filter

for grid, parcels in zip([grid1, grid2], [parcels1, parcels2]):
    for pc_opts in parcel_color_options:
        fig = plot_network_and_parcels(
            grid, parcels, 
            parcel_time_index=0, **pc_opts)
        plt.show()

In [ ]:
parcel_size_norm = Normalize(0, 1)
parcel_size_norm2=colors.LogNorm(vmin=0.01, vmax=1)

parcel_size_options = [
    {# empty dictionary = defaults
    },
    {
        "parcel_color":'b', # specify some simple modifications. 
         "parcel_size":5
    },
    {
        "parcel_size_attribute": "D", # use a parcel attribute. 
        "parcel_size_norm": parcel_color_norm,
        "parcel_size_attribute_title":"Diameter [m]",
        "parcel_alpha":1.0,
    },
    {
        "parcel_size_attribute": "D", # an 
        "parcel_size_norm": parcel_color_norm2,
        "parcel_size_min": 10,
        "parcel_size_max": 100,
    },
]

#add parcel filter

for grid, parcels in zip([grid1, grid2], [parcels1, parcels2]):
    for ps_opts in parcel_size_options:
        fig = plot_network_and_parcels(
            grid, parcels, 
            parcel_time_index=0, **ps_opts)
        plt.show()


In [ ]:
# you can filter parcels! Here we show every 50. 
parcel_filter = np.zeros((parcels2.dataset.dims["item_id"]), dtype=bool)
parcel_filter[::50] = True
pc_opts= {
        "parcel_color_attribute": "D", # a more complex normalization and a parcel filter. 
        "parcel_color_norm": parcel_color_norm2,
        "parcel_color_attribute_title":"Diameter [m]",
        "parcel_alpha": 1.0,
        "parcel_filter": parcel_filter
    }
fig = plot_network_and_parcels(
grid2, parcels2, 
parcel_time_index=0, **pc_opts)
plt.show()

In [ ]:
# you can combine network, parcel size and color options. 

parcel_color_norm=colors.LogNorm(vmin=0.01, vmax=1)

parcel_filter = np.zeros((parcels2.dataset.dims["item_id"]), dtype=bool)
parcel_filter[::10] = True

fig = plot_network_and_parcels(grid2, 
                               parcels2,  
                               parcel_time_index=0, 
                               parcel_filter=parcel_filter,
                               link_attribute="sediment_total_volume", 
                               network_norm=network_norm, 
                               parcel_alpha=1.0, 
                               parcel_size_attribute="D",
                               parcel_color_attribute="D",
                               parcel_color_norm=parcel_color_norm2, 
                               parcel_size_norm=parcel_size_norm,
                               parcel_size_attribute_title="D")